In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
import os
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
from itertools import product, combinations
import math
import warnings
# Import some libraries
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import georasters as gr

In [2]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Tel Aviv']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)

park_entry_point_merge = 0 # ! Be careful with siplyfying topology with this, can alter the scores quite a bit.
grid_block_size = 100 # in meters2 rounded to 100m
nn_iter = 10 # tries to find nearest nodes at route finding
#block_combinations = 100
block_network = 250000 # chunks for route finding

In [3]:
# Block 1 Extract city boundaries, download grids in cities' country

warnings.filterwarnings("ignore")
bound_df = ox.geocoder.geocode_to_gdf(cities)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')

start_time = time.time()

# Get unique ISO countries, so all country-grids are only loaded once (i.e. Philadelphia, Denver and Washington DC in the USA)
C = pd.DataFrame(Countries)
start_time = time.time()
iso_countries = []
print('required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)')
for i in range(len(cities)): 
    countries = bound_df['display_name'][i].rsplit(',')[-1][1:]
    iso = C[C['name'] == countries].iloc[0,1]
    if iso not in iso_countries:
        iso_countries.append(iso)
        
        # Search for data
        products = Product(iso)
        Results = products.description_contains('people per grid-cell 2020')
        list1 = []
        for p in Results:
            prints = '%s/%s\t%s\t%s' % (p.idx, p.country_name,p.dataset_name,p.path)
            list1.append(prints)
        print('wpgpDownload download -i',iso,'--id',list1[0].split("\t")[0].split('/')[0])

print('')
blocks = []
for i in iso_countries:
    # Get the country by saved path ()
    # You can also use sys.path: path = sys.path[0] +'\\' + str('ISL').lower() + '_ppp_2020.tif'
    # The sys.path is the path Python saves the python file automatically before you
    path = 'D:\\Dumps\\WorldPoP_Grids\\' + i.lower() + '_ppp_2020.tif'
    block = gr.from_file(path)
    blocks.append(block)
    print(i,'extracted', round((time.time() - start_time)/60,2),'mns')
bound_df

required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)
wpgpDownload download -i ISR --id 5089

ISR extracted 0.01 mns


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((34.73913 32.03376, 34.74220 32.03297...",32.146977,32.029344,34.852262,34.739131,328372020,relation,1382494,32.0853,34.781806,"Tel Aviv-Yafo, Tel Aviv Subdistrict, Tel Aviv ...",boundary,administrative,0.849988


In [4]:
# Block 2 population grids extraction
start_time = time.time()
clips = []
grids = []
print(str(grid_block_size) + 'm resolution grids extraction')
for i in range(len(cities)):
    iso = C[bound_df['display_name'][i].rsplit(',')[-1][1:] == C['name']].iloc[0,1]
    contains = [j for j, x in enumerate(iso_countries) if x == iso][0]
    
    # Clip the city from the country
    clipped = blocks[contains].clip(bound_df['geometry'][i])
    clipped = clipped[0].to_geopandas()
    
    # Get dissolvement_key for dissolvement. 
    clipped['row3'] = np.floor(clipped['row']/(grid_block_size/100)).astype(int)
    clipped['col3'] = np.floor(clipped['col']/(grid_block_size/100)).astype(int)
    clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)
    clips.append(clipped)
        
    # Dissolve into block by block grids
    popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

    # Get those grids populations and area. Only blocks with population and full blocks
    popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
    popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
    popgrid = popgrid[popgrid['population'] > 0]
    popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]
    
    # Get centroids and coords
    popgrid['centroid'] = popgrid['geometry'].centroid
    popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    popgrid = popgrid.reset_index()
    
    minx = popgrid.bounds['minx']
    maxx = popgrid.bounds['maxx']
    miny = popgrid.bounds['miny']
    maxy = popgrid.bounds['maxy']

    # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
    # I therefore create one polygon
    Poly = []
    for k in range(len(popgrid)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    popgrid['geometry'] = gpd.GeoSeries(Poly, crs = 4326)
    
    #create the buffer of each grid cell, which equals the distance threshold
    popgrid['buffer'] = popgrid['geometry'].to_crs(3043).buffer(max(thresholds))
    
    grids.append(popgrid)
    
    print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    
grids[0]

100m resolution grids extraction
Tel Aviv 0.03 mns


,dissolve_key,geometry,row3,col3,population,area_m,centroid,centroid_m,grid_lon,grid_lat,buffer
0,1-63,"POLYGON ((34.79125 32.14625, 34.79208 32.14625...",1,63,18,9071,POINT (34.79167 32.14583),POINT (3562233.440 4039390.777),3.562233e+06,4.039391e+06,"POLYGON ((3561225.967 4039112.494, 3561199.807..."
1,1-64,"POLYGON ((34.79208 32.14625, 34.79292 32.14625...",1,64,20,9071,POINT (34.79250 32.14583),POINT (3562316.861 4039418.323),3.562317e+06,4.039418e+06,"POLYGON ((3561309.389 4039140.029, 3561283.229..."
2,1-65,"POLYGON ((34.79292 32.14625, 34.79375 32.14625...",1,65,19,9071,POINT (34.79333 32.14583),POINT (3562400.281 4039445.869),3.562400e+06,4.039446e+06,"POLYGON ((3561392.812 4039167.566, 3561366.651..."
3,1-66,"POLYGON ((34.79375 32.14625, 34.79458 32.14625...",1,66,22,9071,POINT (34.79417 32.14583),POINT (3562483.702 4039473.416),3.562484e+06,4.039473e+06,"POLYGON ((3561476.235 4039195.103, 3561450.073..."
4,1-67,"POLYGON ((34.79458 32.14625, 34.79542 32.14625...",1,67,19,9072,POINT (34.79500 32.14583),POINT (3562567.123 4039500.964),3.562567e+06,4.039501e+06,"POLYGON ((3561559.659 4039222.642, 3561533.496..."
...,...,...,...,...,...,...,...,...,...,...,...
7360,99-75,"POLYGON ((34.80125 32.06458, 34.80208 32.06458...",99,75,72,9084,POINT (34.80167 32.06417),POINT (3566405.730 4030110.029),3.566406e+06,4.030110e+06,"POLYGON ((3565398.047 4029832.305, 3565371.940..."
7361,99-76,"POLYGON ((34.80208 32.06458, 34.80292 32.06458...",99,76,72,9085,POINT (34.80250 32.06417),POINT (3566489.264 4030137.560),3.566489e+06,4.030138e+06,"POLYGON ((3565481.584 4029859.827, 3565455.476..."
7362,99-77,"POLYGON ((34.80292 32.06458, 34.80375 32.06458...",99,77,72,9085,POINT (34.80333 32.06417),POINT (3566572.799 4030165.092),3.566573e+06,4.030165e+06,"POLYGON ((3565565.121 4029887.350, 3565539.013..."
7363,99-78,"POLYGON ((34.80375 32.06458, 34.80458 32.06458...",99,78,72,9085,POINT (34.80417 32.06417),POINT (3566656.334 4030192.626),3.566656e+06,4.030193e+06,"POLYGON ((3565648.658 4029914.873, 3565622.549..."


In [5]:
# Block 3 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
    #graphs.append(graph)
    
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    #road_nodes.append(road_node)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    #road_edges.append(road_edge)
    
    # Exclude highways and ramps on edges    
    edge_filter = road_edge[(road_edge['highway'].str.contains('motorway') | 
          (road_edge['highway'].str.contains('trunk') & 
           road_edge['maxspeed'].astype(str).str.contains(
               '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
    road_edges.append(edge_filter)
    
    # Exclude isolated nodes
    fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
    newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
    road_nodes.append(newnodes)
    
    # Get only necessary road connections columns for network performance
    road_con = edge_filter[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    
    # formatting to graph again.
    newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
    edge_filter = edge_filter.set_index(['from','to','keys'])
    edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]
    graph2 = ox.graph_from_gdfs(newnodes, edge_filter)
                            
    graphs.append(graph2)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    
road_edges[0]

Tel Aviv done 0.49 mns


,from,to,keys,osmid,oneway,name,highway,length,geometry,maxspeed,...,ref,bridge,tunnel,access,junction,width,service,est_width,geometry_m,key
0,139693,5723720351,0,5118378,True,ויצמן,tertiary,4.370,"LINESTRING (34.79057 32.09384, 34.79057 32.09388)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564142.594 4033236.134, 3564141.0...",139693-5723720351
1,139693,518158039,0,213434785,False,יהודה המכבי,tertiary,14.108,"LINESTRING (34.79057 32.09384, 34.79042 32.09385)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564142.594 4033236.134, 3564127.3...",139693-518158039
2,139693,139698,0,167691710,True,יהודה המכבי,tertiary,62.173,"LINESTRING (34.79057 32.09384, 34.79063 32.093...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564142.594 4033236.134, 3564148.8...",139693-139698
3,139698,5723720350,0,34747990,False,הרב עמיאל,footway,4.459,"LINESTRING (34.79123 32.09387, 34.79122 32.09391)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564207.460 4033261.383, 3564205.0...",139698-5723720350
4,139698,558854656,0,167691710,True,יהודה המכבי,tertiary,52.222,"LINESTRING (34.79123 32.09387, 34.79178 32.09391)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564207.460 4033261.383, 3564261.4...",139698-558854656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67168,9792512031,34651074,0,649560905,False,הגדוד העברי,residential,6.628,"LINESTRING (34.77792 32.05906, 34.77786 32.05909)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564223.482 4028724.958, 3564216.1...",9792512031-34651074
67169,9792512031,4475742674,0,649560905,False,הגדוד העברי,residential,10.571,"LINESTRING (34.77792 32.05906, 34.77802 32.05902)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564223.482 4028724.958, 3564235.1...",9792512031-4475742674
67170,9798069809,8957012981,0,968151934,False,NaN,cycleway,1.982,"LINESTRING (34.80772 32.11985, 34.80774 32.11984)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564850.222 4036863.602, 3564852.4...",9798069809-8957012981
67171,9798069809,597659121,0,1067092561,False,NaN,cycleway,31.622,"LINESTRING (34.80772 32.11985, 34.80759 32.11959)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (3564850.222 4036863.602, 3564847.1...",9798069809-597659121


In [6]:
# Block 4 city greenspace
start_time = time.time()
parks_in_range = list()
for i in cities:
    gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
    gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
    greenspace = gdf.reset_index()    
    warnings.filterwarnings("ignore")
    
    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')
    
    # Exclude parks below 0.04 ha.
    parks['park_area'] = parks.to_crs(3043).area
    parks = parks[parks['park_area'] > min_park_size]
    print(i, 'done', round((time.time() - start_time) / 60,2),'mns')
    parks = parks.reset_index()
    parks_in_range.append(parks)
    #print(i.rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
parks_in_range[0]

Tel Aviv done 0.01 mns


,components,geometry,element_type,osmid,leisure,name,name:en,name:he,addr:housenumber,addr:street,...,addr:postcode,name:eng,name:en2,name:en3,FIXME,ways,type,geometry_w_buffer,geom buffer diff,park_area
0,0,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",way,24893732,park,ככר מסריק,Masaryk square,ככר מסריק,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.77887 32.07899, 34.77907 32.07885...","POLYGON ((34.77907 32.07885, 34.77909 32.07884...",2805.048014
1,1,"POLYGON ((34.78793 32.08120, 34.78719 32.08127...",way,26301651,park,גן אריסון,Arison,גן אריסון,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.78817 32.08118, 34.78812 32.08056...","POLYGON ((34.78812 32.08056, 34.78811 32.08054...",6150.153590
2,2,"MULTIPOLYGON (((34.78474 32.08178, 34.78451 32...",way,26301668,park,Sutin grove,Sutin grove,חורשת סוטין,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.78635 32.08181, 34.78568 32.08192...","POLYGON ((34.78568 32.08192, 34.78566 32.08192...",14160.961618
3,3,"MULTIPOLYGON (((34.75085 32.03107, 34.75062 32...",way,28960146,park,None,None,None,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.74727 32.03183, 34.74728 32.03193...","POLYGON ((34.74728 32.03193, 34.74728 32.03195...",36491.543077
4,4,"POLYGON ((34.74635 32.03353, 34.74594 32.03318...",way,28960238,park,None,None,None,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.74645 32.03371, 34.74704 32.03346...","POLYGON ((34.74704 32.03346, 34.74741 32.03335...",4718.232354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,397,"MULTIPOLYGON (((34.79678 32.07276, 34.79665 32...",way,992277250,park,None,None,None,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.79602 32.07332, 34.79603 32.07334...","POLYGON ((34.79603 32.07334, 34.79604 32.07336...",1216.734414
391,398,"MULTIPOLYGON (((34.78580 32.11097, 34.78576 32...",way,1003023656,park,None,None,None,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.78523 32.10960, 34.78521 32.10960...","POLYGON ((34.78521 32.10960, 34.78519 32.10960...",22652.648573
392,399,"POLYGON ((34.79862 32.05882, 34.79862 32.05860...",way,1011503054,park,None,None,None,None,None,...,None,None,None,None,yes,None,None,"POLYGON ((34.79862 32.05902, 34.79912 32.05901...","POLYGON ((34.79912 32.05901, 34.79914 32.05901...",1378.493178
393,400,"POLYGON ((34.80261 32.07735, 34.80329 32.07753...",way,1014602413,park,Petai,None,פטאי,None,None,...,None,None,None,None,None,None,None,"POLYGON ((34.80238 32.07730, 34.80237 32.07732...","POLYGON ((34.80237 32.07732, 34.80237 32.07734...",2224.365365


In [7]:
# Block 5 park entry points
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        # Check distance of roads and UGS-edge
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        # Get nodes within 25m as park entry points
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry','park_area']], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    #ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Tel Aviv 0.0 % done 0.0  mns
Tel Aviv 12.7 % done 0.1  mns
Tel Aviv 25.3 % done 0.18  mns
Tel Aviv 38.0 % done 0.27  mns
Tel Aviv 50.6 % done 0.35  mns
Tel Aviv 63.3 % done 0.43  mns
Tel Aviv 75.9 % done 0.51  mns
Tel Aviv 88.6 % done 0.59  mns
Tel Aviv 100 % done 0.7  mns


,osmid,y,x,highway,street_count,ref,geometry_x,geometry_m,osmid_var,Park_No,park_lon,park_lat,geometry_y,park_area,park_size_walkable,walk_area,share_walked
0,267816148,32.077971,34.778273,NaN,4,NaN,POINT (34.77827 32.07797),POINT (3563525.766 4030962.337),267816148,0,3.563526e+06,4.030962e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0
1,332205122,32.078866,34.778701,NaN,3,NaN,POINT (34.77870 32.07887),POINT (3563533.930 4031081.887),332205122,0,3.563534e+06,4.031082e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0
2,332205125,32.078678,34.778984,NaN,3,NaN,POINT (34.77898 32.07868),POINT (3563569.558 4031069.030),332205125,0,3.563570e+06,4.031069e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0
3,412518614,32.078828,34.778648,NaN,3,NaN,POINT (34.77865 32.07883),POINT (3563530.105 4031075.591),412518614,0,3.563530e+06,4.031076e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0
4,441130038,32.078808,34.778838,NaN,3,NaN,POINT (34.77884 32.07881),POINT (3563549.897 4031079.580),441130038,0,3.563550e+06,4.031080e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,5225421573,32.070153,34.799474,NaN,4,NaN,POINT (34.79947 32.07015),POINT (3565953.703 4030742.216),5225421573,394,3.565954e+06,4.030742e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0
3889,9666719933,32.067273,34.798740,NaN,3,NaN,POINT (34.79874 32.06727),POINT (3565991.887 4030378.953),9666719933,394,3.565992e+06,4.030379e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0
3890,9666719934,32.067270,34.798780,NaN,4,NaN,POINT (34.79878 32.06727),POINT (3565996.018 4030380.041),9666719934,394,3.565996e+06,4.030380e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0
3891,9666719935,32.067260,34.798910,NaN,4,NaN,POINT (34.79891 32.06726),POINT (3566009.452 4030383.111),9666719935,394,3.566009e+06,4.030383e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0


In [8]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,highway,street_count,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_area,park_size_walkable,walk_area,share_walked,geometry_m_buffer,components,comp_centroid,centr_dist
0,267816148,32.077971,34.778273,NaN,4,NaN,POINT (34.77827 32.07797),POINT (3563525.766 4030962.337),267816148,0,...,4.030962e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0,POLYGON EMPTY,0,POINT (34.77827 32.07797),0.0
1,332205122,32.078866,34.778701,NaN,3,NaN,POINT (34.77870 32.07887),POINT (3563533.930 4031081.887),332205122,0,...,4.031082e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0,POLYGON EMPTY,1,POINT (34.77870 32.07887),0.0
2,332205125,32.078678,34.778984,NaN,3,NaN,POINT (34.77898 32.07868),POINT (3563569.558 4031069.030),332205125,0,...,4.031069e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0,POLYGON EMPTY,2,POINT (34.77898 32.07868),0.0
3,412518614,32.078828,34.778648,NaN,3,NaN,POINT (34.77865 32.07883),POINT (3563530.105 4031075.591),412518614,0,...,4.031076e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0,POLYGON EMPTY,3,POINT (34.77865 32.07883),0.0
4,441130038,32.078808,34.778838,NaN,3,NaN,POINT (34.77884 32.07881),POINT (3563549.897 4031079.580),441130038,0,...,4.031080e+06,"POLYGON ((34.77872 32.07884, 34.77808 32.07820...",2805.048014,"POLYGON ((34.77892 32.07870, 34.77828 32.07806...",2805.048014,1.0,POLYGON EMPTY,4,POINT (34.77884 32.07881),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,5225421573,32.070153,34.799474,NaN,4,NaN,POINT (34.79947 32.07015),POINT (3565953.703 4030742.216),5225421573,394,...,4.030742e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0,POLYGON EMPTY,3888,POINT (34.79947 32.07015),0.0
3889,9666719933,32.067273,34.798740,NaN,3,NaN,POINT (34.79874 32.06727),POINT (3565991.887 4030378.953),9666719933,394,...,4.030379e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0,POLYGON EMPTY,3889,POINT (34.79874 32.06727),0.0
3890,9666719934,32.067270,34.798780,NaN,4,NaN,POINT (34.79878 32.06727),POINT (3565996.018 4030380.041),9666719934,394,...,4.030380e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0,POLYGON EMPTY,3890,POINT (34.79878 32.06727),0.0
3891,9666719935,32.067260,34.798910,NaN,4,NaN,POINT (34.79891 32.06726),POINT (3566009.452 4030383.111),9666719935,394,...,4.030383e+06,"POLYGON ((34.79954 32.07011, 34.79946 32.06994...",2714.260017,"POLYGON ((34.79952 32.06995, 34.79950 32.06986...",2714.260017,1.0,POLYGON EMPTY,3891,POINT (34.79891 32.06726),0.0


In [9]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    #blockA = block_combinations
    print(cities[l])
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    
    # Reduce the size of combinations per iteration
    len4 = 1
    len5 = len1 * len2
    blockC = len5
    while blockC > 10000000:
        blockC = len5 / len4
        #print(blockC, len4)
        len4 = len4+1
        
    # Amount of grids taken per iteration block
    block = round(len1 / len4)
    
    output = pd.DataFrame()
    len_mat = 0
    len_df = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len4):
        # Check all grid-park combinations per block
        l1, l2 = range(i*block,(i+1)*block), range(0,len2)
        listed = pd.DataFrame(list(product(l1, l2)))
                
        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                             'share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])

        # Get the euclidean distances
        mat = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))        
        
        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat)[np.array(mat) <= np.max(thresholds)]

        # join the other gravity euclidean scores and other information
        mat_df.columns = ['Euclidean'] 
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])
        len_mat = len_mat + len(mat_df)
        len_df = len_df + len(node_merged)
            
        if ( (i+1) % 1 == 0) | ((i+1) == len4): 
            print((i+1),'/',len4,'comb. done',round((time.time() - start_time) / 60,2),' mns')
            
            print('of',len_df,'within a Gravity model variant in one of',thresholds,'m threshold:',len_mat)
            len_mat = 0
            len_df = 0
    print('total combinations',len(output))  
    # Renaming columns
    output.columns = ['Euclidean','Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','park_lon',
                      'park_lat','parkshare_walked','park_area','walk_area_m2']
    
    output = output[['Euclidean','Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid',
                     'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','park_area']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        try:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        except:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    #output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
    #                                               ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]    

Tel Aviv
1 / 4 comb. done 0.3  mns
of 7167013 within a Gravity model variant in one of [300, 600, 1000] m threshold: 221868
2 / 4 comb. done 0.67  mns
of 7167013 within a Gravity model variant in one of [300, 600, 1000] m threshold: 258024
3 / 4 comb. done 0.99  mns
of 7167013 within a Gravity model variant in one of [300, 600, 1000] m threshold: 263214
4 / 4 comb. done 1.29  mns
of 7167013 within a Gravity model variant in one of [300, 600, 1000] m threshold: 284885
0.0 % gridentry done 1.29  mns
24.3 % gridentry done 1.65  mns
48.6 % gridentry done 2.04  mns
73.0 % gridentry done 2.42  mns
97.3 % gridentry done 2.81  mns
100 % gridentry done 2.88  mns


,index,Euclidean,Grid_No,Park_entry_No,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,park_area,grid_osm,geometry,geometry_m
0,88680,995.894153,312,48,POINT (34.76083 32.05917),POINT (3562506.440 4028173.682),6,1435925458,POINT (34.76458 32.05179),"POLYGON ((34.76381 32.05197, 34.76273 32.05226...",POINT (3563167.946 4027429.226),39452.230045,4139634042,POINT (34.76097 32.05909),POINT (3562523.530 4028169.072)
1,94203,888.702712,312,51,POINT (34.76083 32.05917),POINT (3562506.440 4028173.682),6,4637004904,POINT (34.76297 32.05223),"POLYGON ((34.76381 32.05197, 34.76273 32.05226...",POINT (3562989.686 4027427.849),39452.230045,4139634042,POINT (34.76097 32.05909),POINT (3562523.530 4028169.072)
2,99726,920.684062,312,54,POINT (34.76083 32.05917),POINT (3562506.440 4028173.682),6,7944620157,POINT (34.76349 32.05209),"POLYGON ((34.76381 32.05197, 34.76273 32.05226...",POINT (3563046.856 4027428.290),39452.230045,4139634042,POINT (34.76097 32.05909),POINT (3562523.530 4028169.072)
3,101567,977.830561,312,55,POINT (34.76083 32.05917),POINT (3562506.440 4028173.682),6,8262904744,POINT (34.76433 32.05186),"POLYGON ((34.76381 32.05197, 34.76273 32.05226...",POINT (3563140.175 4027429.012),39452.230045,4139634042,POINT (34.76097 32.05909),POINT (3562523.530 4028169.072)
4,1233782,285.150464,312,670,POINT (34.76083 32.05917),POINT (3562506.440 4028173.682),35,365768435,POINT (34.76265 32.06087),"POLYGON ((34.76255 32.06087, 34.76284 32.06052...",POINT (3562622.066 4028434.337),11520.579267,4139634042,POINT (34.76097 32.05909),POINT (3562523.530 4028169.072)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027986,6245998,996.648878,6849,3392,POINT (34.79167 32.13917),POINT (3562492.425 4038606.359),325,1546447399,POINT (34.79374 32.13132),"POLYGON ((34.79308 32.13139, 34.79298 32.13110...",POINT (3563004.444 4037751.289),2876.384064,1800488877,POINT (34.79173 32.13880),POINT (3562513.466 4038565.476)
1027987,6247839,965.226692,6849,3393,POINT (34.79167 32.13917),POINT (3562492.425 4038606.359),325,2449309423,POINT (34.79320 32.13149),"POLYGON ((34.79308 32.13139, 34.79298 32.13110...",POINT (3562944.287 4037753.433),2876.384064,1800488877,POINT (34.79173 32.13880),POINT (3562513.466 4038565.476)
1027988,6249680,973.824597,6849,3394,POINT (34.79167 32.13917),POINT (3562492.425 4038606.359),325,5841591608,POINT (34.79335 32.13144),"POLYGON ((34.79308 32.13139, 34.79298 32.13110...",POINT (3562961.275 4037752.829),2876.384064,1800488877,POINT (34.79173 32.13880),POINT (3562513.466 4038565.476)
1027989,6247838,983.766194,6848,3393,POINT (34.79083 32.13917),POINT (3562408.996 4038578.817),325,2449309423,POINT (34.79320 32.13149),"POLYGON ((34.79308 32.13139, 34.79298 32.13110...",POINT (3562944.287 4037753.433),2876.384064,1800488882,POINT (34.79082 32.13893),POINT (3562416.860 4038551.056)


In [10]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Tel Aviv 1027991


In [13]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = block_network

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    # Divide in chunks of block for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        s_mat5 = list([])
        mat_nn = []
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(-1)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
                s_mat5.append(1)
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                    s_mat5.append(0)
                except:
                    # Otherwise the nearest node is taken, which is iterated X times at max, check assumptions, block #0 
                    # Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < nn_iter and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                            s_mat4.append(str(len3)+'grid > n-park')
                            s_mat5.append(1)
                        except:
                            try:
                                alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                                s_mat4.append(str(len3)+'n-park > grid')
                                s_mat5.append(0)
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                    s_mat4.append(str(len3)+'n-grid > park')
                                    s_mat5.append(1)
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                        s_mat4.append(str(len3)+'park > n-grid')
                                        s_mat5.append(0)
                                    except:
                                        if len3 == nn_iter:
                                            #print(i+block*k,i+block*k,
                                            #      'No route between grid and park-entry and their both 10 alternatives')
                                            pass
                                        pass
                    #print(len(alt_route))
                    if len(alt_route) == 0: 
                        alt = alt_route 
                    else: 
                        alt = alt_route[0]
                    len4 = len(alt)
                    #print(len4)
                    #mat_nn = []
                    if len4 > 0:
                        #print('for index',i+block*k,'nearest node found between', 
                        #                       alt[0],'and',alt[-1])
                        mat_nn.append(i+block*k)
                        s_mat.append(alt)
                        shortest_to = list(alt[1:len(alt)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4)))
                        s_mat3.append(list(np.arange(0, len4)))
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')
                        s_mat5.append(2)
                        print(i+block*k,'No route between grid and park-entry and their both',nn_iter,'alternatives')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')
        print('for', len(mat_nn),'routes nearest nodes found')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # dissolve route
        routes2 = routes[['route','geometry']].dissolve('route')
        
        # get used grid- and parkosm. Differs at NN-route.
        route_reset = routes.reset_index()
        origin = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmin()),]
        origin = origin.reset_index().iloc[:,-1]
        dest = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmax()),]
        dest = dest.reset_index().iloc[:,-1]
        
        # grid > park = 1, park > grid = 0, no way = 2, detailed way in way_calc.
        routes2['G1-P0'] = s_mat5
        routes2['real_grid'] = np.where(routes2['G1-P0'] == 1, origin, dest)
        routes2['real_park'] = np.where(routes2['G1-P0'] == 1, dest, origin)
        routes2['way_calc.'] = s_mat4
        
        # get route cost, steps, additional information.
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                              'Grid_m_centroid','Euclidean']],
                                                left_index = True, right_index = True)
        routes2 = pd.merge(routes2, road_nodes[j]['geometry_m'], how = 'left', left_on = 'real_grid', right_index = True)
        
        # calculate distance of used road-entry for grid-centroid.
        routes2['real_G-entry'] = round(gpd.GeoSeries(routes2['Grid_m_centroid'], crs = 3043
                                                       ).distance(routes2['geometry_m']),3)
        
        # Calculcate total route cost for the four gravity variants
        routes2['Tcost'] = routes2['route_cost'] + routes2['real_G-entry']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]

,geometry,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,grid_osm,Park_No,Park_entry_No,Parkroad_osmid,Grid_m_centroid,Euclidean,geometry_m,real_G-entry,Tcost,gridpark_no
index,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((34.76097 32.05909, 34.76092 ...",1,4139634042,1435925458,normal way,1384.087,25,312,4139634042,6,48,1435925458,POINT (3562506.440 4028173.682),995.894153,POINT (3562523.530 4028169.072),17.700,1401.787,312-6
1,"MULTILINESTRING ((34.76097 32.05909, 34.76092 ...",1,4139634042,4637004904,normal way,1352.627,25,312,4139634042,6,51,4637004904,POINT (3562506.440 4028173.682),888.702712,POINT (3562523.530 4028169.072),17.700,1370.327,312-6
2,"MULTILINESTRING ((34.76097 32.05909, 34.76092 ...",1,4139634042,7944620157,normal way,1403.707,26,312,4139634042,6,54,7944620157,POINT (3562506.440 4028173.682),920.684062,POINT (3562523.530 4028169.072),17.700,1421.407,312-6
3,"MULTILINESTRING ((34.76097 32.05909, 34.76092 ...",1,4139634042,8262904744,normal way,1408.899,26,312,4139634042,6,55,8262904744,POINT (3562506.440 4028173.682),977.830561,POINT (3562523.530 4028169.072),17.700,1426.599,312-6
4,"MULTILINESTRING ((34.76097 32.05909, 34.76092 ...",1,4139634042,365768435,normal way,393.059,10,312,4139634042,35,670,365768435,POINT (3562506.440 4028173.682),285.150464,POINT (3562523.530 4028169.072),17.700,410.759,312-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027986,"MULTILINESTRING ((34.79173 32.13880, 34.79170 ...",1,1800488877,1546447399,normal way,1654.988,17,6849,1800488877,325,3392,1546447399,POINT (3562492.425 4038606.359),996.648878,POINT (3562513.466 4038565.476),45.980,1700.968,6849-325
1027987,"MULTILINESTRING ((34.79173 32.13880, 34.79170 ...",1,1800488877,2449309423,normal way,1339.991,16,6849,1800488877,325,3393,2449309423,POINT (3562492.425 4038606.359),965.226692,POINT (3562513.466 4038565.476),45.980,1385.971,6849-325
1027988,"MULTILINESTRING ((34.79173 32.13880, 34.79170 ...",1,1800488877,5841591608,normal way,1346.656,16,6849,1800488877,325,3394,5841591608,POINT (3562492.425 4038606.359),973.824597,POINT (3562513.466 4038565.476),45.980,1392.636,6849-325


In [14]:
# Block 8 nearest entry point per grid and UGS
gp_nearest = []
for i in range(len(cities)):
    gp_nn = Routes[i].iloc[Routes[i].groupby('gridpark_no')['Tcost'].idxmin()]
    gp_nn = gp_nn[gp_nn['Tcost'] <= max(thresholds)]
    gp_nn = pd.merge(gp_nn, grids[i]['population'], left_on='Grid_No', right_index = True)
    gp_nn = pd.merge(gp_nn, parks_in_range[i]['park_area'], left_on = 'Park_No', right_index = True)
    
    gp_nn.index.name = 'idx'
    gp_nn = gp_nn.sort_values('idx')
    gp_nn = gp_nn.reset_index()
    gp_nearest.append(gp_nn)
gp_nearest[0].sort_values('Grid_No')

,idx,geometry,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,grid_osm,...,Park_entry_No,Parkroad_osmid,Grid_m_centroid,Euclidean,geometry_m,real_G-entry,Tcost,gridpark_no,population,park_area
9721,177984,"MULTILINESTRING ((34.79745 32.13726, 34.79781 ...",1,8908951818,8035408776,normal way,823.003,5,14,8908951818,...,1597,8035408776,POINT (3563025.382 4038673.576),893.495037,POINT (3563145.601 4038573.612),156.350,979.353,14-99,20,5759.157022
9722,177985,"MULTILINESTRING ((34.79745 32.13726, 34.79781 ...",1,8908951818,8035408776,normal way,823.003,5,15,8908951818,...,1597,8035408776,POINT (3563108.814 4038701.124),920.389979,POINT (3563145.601 4038573.612),132.712,955.715,15-99,26,5759.157022
9727,178021,"MULTILINESTRING ((34.79821 32.13741, 34.79815 ...",1,9112391174,8035408776,normal way,843.672,6,16,9112391174,...,1597,8035408776,POINT (3563192.246 4038728.674),954.643155,POINT (3563215.520 4038616.287),114.771,958.443,16-99,22,5759.157022
6461,115048,"MULTILINESTRING ((34.75978 32.06340, 34.75975 ...",1,1184173335,1203435228,normal way,223.473,2,19,1184173335,...,2246,1203435228,POINT (3562261.555 4028636.624),130.384831,POINT (3562237.114 4028637.791),24.469,247.942,19-169,43,143423.401215
6459,115010,"MULTILINESTRING ((34.75978 32.06340, 34.75975 ...",1,1184173335,681556997,normal way,753.859,12,19,1184173335,...,671,681556997,POINT (3562261.555 4028636.624),391.147482,POINT (3562237.114 4028637.791),24.469,778.328,19-35,43,11520.579267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46095,944800,"MULTILINESTRING ((34.80425 32.06418, 34.80343 ...",1,2299865133,1329719,normal way,815.890,10,7363,2299865133,...,3569,1329719,POINT (3566656.334 4030192.626),788.780575,POINT (3566663.751 4030196.814),8.517,824.407,7363-359,72,4400.746173
46094,944789,"MULTILINESTRING ((34.80425 32.06418, 34.80343 ...",1,2299865133,2088696781,normal way,857.409,13,7363,2299865133,...,3262,2088696781,POINT (3566656.334 4030192.626),687.000722,POINT (3566663.751 4030196.814),8.517,865.926,7363-308,72,5238.763563
46093,944765,"LINESTRING (34.80425 32.06418, 34.80343 32.063...",1,2299865133,3497181257,normal way,86.205,1,7363,2299865133,...,3030,3497181257,POINT (3566656.334 4030192.626),88.050004,POINT (3566663.751 4030196.814),8.517,94.722,7363-277,72,1707.250759
46092,944758,"MULTILINESTRING ((34.80425 32.06418, 34.80343 ...",1,2299865133,4289060558,normal way,962.609,8,7363,2299865133,...,2939,4289060558,POINT (3566656.334 4030192.626),405.350817,POINT (3566663.751 4030196.814),8.517,971.126,7363-261,72,12301.180238


In [15]:
parks_in_range[0]['geometry'].to_crs(3043).area

0       2805.048014
1       6150.153590
2      14160.961618
3      36491.543077
4       4718.232354
           ...     
390     1216.734414
391    22652.648573
392     1378.493178
393     2224.365365
394     2714.260017
Length: 395, dtype: float64

In [16]:
v = -thresholds[j]**2/np.log(0.5)
# use gussian distribution: let v= 923325, then the weight for 800m is 0.5
# add a column of weight: apply the decay function on distance
gp_nearest[i]['weight'] = np.exp(-(gp_nearest[i]['Tcost']**2/v)).astype(float)
gp_nearest[i]['pop_weight'] = gp_nearest[i]['weight'] * gp_nearest[i]['population']
        
# get the sum of weighted population each green space has to serve.
s_w_p = pd.DataFrame(gp_nearest[i].groupby('Park_No').sum('pop_weight')['pop_weight'])
        
# delete other columns, because they are useless after groupby
s_w_p = s_w_p.rename({'pop_weight':'pop_weight_sum'},axis = 1)
middle = pd.merge(gp_nearest[i],s_w_p, how = 'left', on = 'Park_No' )
# calculate the supply-demand ratio for each green space
middle['green_supply'] = middle['park_area']/middle['pop_weight_sum']
        
# caculate the accessbility score for each green space that each population grid cell could reach
middle['Sc-access'] = middle['weight'] * middle['green_supply']
# add the scores for each population grid cell
pop_score_df = pd.DataFrame(middle.groupby('Grid_No').sum('Sc-access')['Sc-access'])
        
# calculate the mean distance of all the green space each population grid cell could reach
mean_dist = middle.groupby('Grid_No').mean('Tcost')['Tcost']
pop_score_df['M-dist'] = mean_dist
        
# calculate the mean area of all the green space each population grid cell could reach
mean_area = middle.groupby('Grid_No').mean('park_area')['park_area']
pop_score_df['M-area'] = mean_area

# calculate the mean supply_demand ratio of all the green space each population grid cell could reach
mean_supply = middle.groupby('Grid_No').mean('green_supply')['green_supply']
pop_score_df['M-supply'] = mean_supply
        
pop_score = pop_score_df
        
pop_score_df = pop_score_df.join(grids[i]['population'], how = 'right')
pop_score_df['Sc-norm'] = pop_score_df['Sc-access'] / pop_score_df['population']

pop_score_df = pop_score_df.loc[:, pop_score_df.columns != 'population']
pop_score_df = pop_score_df.add_suffix(' '+str(thresholds[j]))
pop_score['Sc-access']

Grid_No
14       0.006730
15       0.009572
16       0.009195
19      31.285795
20      43.328625
          ...    
7359     8.453303
7360     4.665609
7361     4.471683
7362     6.740723
7363     7.043375
Name: Sc-access, Length: 7085, dtype: float64

In [18]:
# Calculating the M2sfca score.
M2SFCA_cities = []
for i in range(len(cities)):
    M2SFCA_score = grids[i][['population','geometry']]
    for j in range(len(thresholds)):
        v = -thresholds[j]**2/np.log(0.5)
        # use gussian distribution: let v= 923325, then the weight for 800m is 0.5
        # add a column of weight: apply the decay function on distance
        gp_nearest[i]['weight'] = np.exp(-(gp_nearest[i]['Tcost']**2/v)).astype(float)
        gp_nearest[i]['pop_weight'] = gp_nearest[i]['weight'] * gp_nearest[i]['population']
        
        # get the sum of weighted population each green space has to serve.
        s_w_p = pd.DataFrame(gp_nearest[i].groupby('Park_No').sum('pop_weight')['pop_weight'])
        
        # delete other columns, because they are useless after groupby
        s_w_p = s_w_p.rename({'pop_weight':'pop_weight_sum'},axis = 1)
        middle = pd.merge(gp_nearest[i],s_w_p, how = 'left', on = 'Park_No' )
        # calculate the supply-demand ratio for each green space
        middle['green_supply'] = middle['park_area']/middle['pop_weight_sum']
        
        # caculate the accessbility score for each green space that each population grid cell could reach
        middle['Sc-access'] = middle['weight'] * middle['green_supply']
        # add the scores for each population grid cell
        pop_score_df = pd.DataFrame(middle.groupby('Grid_No').sum('Sc-access')['Sc-access'])
        
        # calculate the mean distance of all the green space each population grid cell could reach
        mean_dist = middle.groupby('Grid_No').mean('Tcost')['Tcost']
        pop_score_df['M-dist'] = mean_dist
        
        # calculate the mean area of all the green space each population grid cell could reach
        mean_area = middle.groupby('Grid_No').mean('park_area')['park_area']
        pop_score_df['M-area'] = mean_area

        # calculate the mean supply_demand ratio of all the green space each population grid cell could reach
        mean_supply = middle.groupby('Grid_No').mean('green_supply')['green_supply']
        pop_score_df['M-supply'] = mean_supply
        
        pop_score = pop_score_df
        
        pop_score_df = pop_score_df.join(grids[i]['population'], how = 'right')
        pop_score_df['Sc-norm'] = pop_score_df['Sc-access'] / pop_score_df['population']
        
        pop_score_df = pop_score_df.loc[:, pop_score_df.columns != 'population']
        pop_score_df = pop_score_df.add_suffix(' '+str(thresholds[j]))
        M2SFCA_score = M2SFCA_score.join(pop_score_df, how = 'left')
        
    M2SFCA_cities.append(M2SFCA_score)
    M2SFCA_score.loc[:, M2SFCA_score.columns != 'geometry'].to_csv('D:/Dumps/M2SFCA-OD/Scores'+cities[i]+'.csv')
M2SFCA_cities[0]

,population,geometry,Sc-access 300,M-dist 300,M-area 300,M-supply 300,Sc-norm 300,Sc-access 600,M-dist 600,M-area 600,M-supply 600,Sc-norm 600,Sc-access 1000,M-dist 1000,M-area 1000,M-supply 1000,Sc-norm 1000
0,18,"POLYGON ((34.79125 32.14625, 34.79208 32.14625...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20,"POLYGON ((34.79208 32.14625, 34.79292 32.14625...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19,"POLYGON ((34.79292 32.14625, 34.79375 32.14625...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,"POLYGON ((34.79375 32.14625, 34.79458 32.14625...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,"POLYGON ((34.79458 32.14625, 34.79542 32.14625...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7360,72,"POLYGON ((34.80125 32.06458, 34.80208 32.06458...",4.665609,812.356818,8802.553880,3.866730,0.064800,5.044701,812.356818,8802.553880,1.304352,0.070065,6.059496,812.356818,8802.553880,0.835597,0.084160
7361,72,"POLYGON ((34.80208 32.06458, 34.80292 32.06458...",4.471683,701.643143,28261.163762,10.827573,0.062107,9.872971,701.643143,28261.163762,3.710725,0.137125,11.958989,701.643143,28261.163762,2.459318,0.166097
7362,72,"POLYGON ((34.80292 32.06458, 34.80375 32.06458...",6.740723,675.458000,25335.393490,9.969700,0.093621,11.868942,675.458000,25335.393490,3.392738,0.164846,13.117719,675.458000,25335.393490,2.241136,0.182191
7363,72,"POLYGON ((34.80375 32.06458, 34.80458 32.06458...",7.043375,675.661143,28261.163762,10.827573,0.097825,12.775918,675.661143,28261.163762,3.710725,0.177443,13.181833,675.661143,28261.163762,2.459318,0.183081


In [19]:
print(pop_score['Sc-access'].mean(),
      pop_score['Sc-access'].max(),
      pop_score['Sc-access'].min(),
      pop_score['Sc-access'].median(),
      np.percentile(pop_score['Sc-access'], 25),
      np.percentile(pop_score['Sc-access'], 75)
     )

24.440751565626282 588.4819190351076 0.16298320121953425 14.728406913988284 7.7030336140279765 26.020454385665378


In [20]:
print(round((time.time() - start) / 60,2),'mns')

96.13 mns
